## Imports

In [315]:
import numpy as np
import pandas as pd 
import os as os
from time import time

from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder, LabelEncoder

import warnings
warnings.filterwarnings('ignore')

## Create Classes for Data Processing

In [473]:
class Load_Data(BaseEstimator, TransformerMixin):
    def __init__(self, features=None):
        self.features = features
        self.weather_dir = ''
        self.soil_dir = ''
        self.drop_columns = ['STATION', 'NAME', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'AWND_ATTRIBUTES', 'PGTM_ATTRIBUTES', 
                             'PSUN', 'PSUN_ATTRIBUTES', 'SNOW', 'SNOW_ATTRIBUTES', 'SNWD', 'SNWD_ATTRIBUTES', 'TAVG',
                             'TAVG_ATTRIBUTES', 'TMAX_ATTRIBUTES', 'TMIN_ATTRIBUTES', 'TSUN', 'TSUN_ATTRIBUTES', 'WDF2_ATTRIBUTES', 
                             'WDF5_ATTRIBUTES', 'WSF2_ATTRIBUTES','WSF5_ATTRIBUTES', 'WT01_ATTRIBUTES', 'WT02_ATTRIBUTES', 
                             'WT03_ATTRIBUTES', 'WT06_ATTRIBUTES', 'WT08_ATTRIBUTES', 'PRCP_ATTRIBUTES']
        
    def fit(self, w_dir, s_dir):
        self.weather_dir = w_dir
        self.soil_dir = s_dir
        return self
    
    def transform(self, X):
        #Aggregate all 43 files into one file
        file_list = os.listdir(self.soil_dir)
        agg_data = pd.DataFrame()
        for file in file_list:
            path = self.soil_dir + file
            curr_data = pd.read_csv(path, sep='\t')
            agg_data = agg_data.append(curr_data)
        
        #Drop rows with only NAs for measurement values
        soil = agg_data.dropna(thresh=10)
        
        #Import weather files and drop unnessecary fields
        weather = pd.read_csv(self.weather_dir)
        drop_cols = list(set(weather.columns).intersection(self.drop_columns))
        weather = weather.drop(columns = self.drop_columns)
        
        #Convert both files to use same datetime
        soil['Date'] = pd.to_datetime(soil['Date'])
        weather['DATE'] = pd.to_datetime(weather['DATE'])
        
        #Join previous 10 days weather to moisture readings
        for i in range(0, 11):
            weather_new = weather.add_suffix('_' + str(i))
            soil = soil.merge(weather_new, how = 'left', left_on = 'Date', right_on = weather['DATE'] - pd.DateOffset(i * -1))
        
        #Setting location to category for faster processing
        soil['Location'] = soil['Location'].astype('category')
            
        return soil

In [474]:
class Feature_Engineer(BaseEstimator, TransformerMixin):
    def __init__(self, features=None):
        self.features = features
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        #Add categorical feature that simply stores if it rained that day or not
        for i in range(11):
            col_name = 'PRCP_' + str(i)
            rain_y_n_name = 'RAIN_Y_N_' + str(i)
            X[rain_y_n_name] = np.nan
            X[rain_y_n_name].loc[X[col_name] > 0] = 1
            X[rain_y_n_name].loc[X[col_name] == 0] = 0
            X[rain_y_n_name] = X[rain_y_n_name].astype('object')
        return X

In [475]:
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        print(X)
        return X[self.attribute_names].values

In [476]:
class Convert_Date(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names = None):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        X['Date'] = pd.to_timedelta(X['Date']).dt.total_seconds().astype(int)
        return X

## Create a Pipeline That Uses Data Processing Class

In [477]:
%%time
soil_file_dir = 'data/soil/'
weather_file_dir = 'data/weather/weather_data.csv'
x = 0

pre_work_pipeline = Pipeline([
    ('prework', Load_Data()),
    ('features', Feature_Engineer())
])

pre_work_pipeline.fit(weather_file_dir, soil_file_dir)
prework_df = pre_work_pipeline.transform(x)
#Save to CSV so that we do not need to import and clean data everytime
prework_df.to_csv('clean_data.csv')

Wall time: 21.4 s


## Split Train and Test

In [478]:
prework_df['Location'] = prework_df['Location'].astype('object')

In [479]:
# First split out y values
y_cols = ['VW_30cm', 'VW_60cm', 'VW_90cm', 'VW_120cm', 'VW_150cm']
x_cols = [col for col in prework_df.columns if col not in y_cols]
X = prework_df.loc[:, x_cols]
y = prework_df.loc[:, y_cols]

# Split training and test data
# 80-20 ratio
# Trying to keep same ratios for each location using stratify

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify = X['Location'], random_state=42)

## Generic Pipeline

In [361]:
num_attribs = X_train.select_dtypes(exclude=['object', 'category', 'datetime64']).columns
cat_attribs = X_train.select_dtypes(include=['object', 'category', 'datetime64']).columns

date_attribs = X_train.select_dtypes(include=['datetime64']).columns
print(cat_attribs)
num_pipeline = Pipeline([
        ('selector', DataFrameSelector(num_attribs)),
        ('imputer', SimpleImputer(strategy='constant', fill_value = 0)),
        ('std_scaler', StandardScaler()),
    ])

cat_pipeline = Pipeline([
        ('selector', DataFrameSelector(cat_attribs)),
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('ohe', OneHotEncoder(sparse=False, handle_unknown="ignore"))
    ])

date_pipeline = Pipeline([
        ('selector', DataFrameSelector(date_attribs)),
        ('converter', Convert_Date())
])

data_prep_pipeline = FeatureUnion(transformer_list=[
        ('cat_pipeline', cat_pipeline),
        #('num_pipeline', num_pipeline),
        #('date_pipeline', date_pipeline)
    ])


Index(['Location', 'RAIN_Y_N_0', 'RAIN_Y_N_1', 'RAIN_Y_N_2', 'RAIN_Y_N_3',
       'RAIN_Y_N_4', 'RAIN_Y_N_5', 'RAIN_Y_N_6', 'RAIN_Y_N_7', 'RAIN_Y_N_8',
       'RAIN_Y_N_9', 'RAIN_Y_N_10'],
      dtype='object')


In [365]:
from sklearn.pipeline import make_union, make_pipeline
from sklearn.preprocessing import FunctionTransformer

def get_text_cols(df):
    return df.select_dtypes(exclude=['object', 'category', 'datetime64']).columns

pipe = make_union(*[
    make_pipeline(FunctionTransformer(get_text_cols, validate = False), SimpleImputer(strategy='constant', fill_value = 0))
])

In [387]:
X_train.dtypes

Location               object
Date           datetime64[ns]
T_30cm                float64
T_60cm                float64
T_90cm                float64
                    ...      
RAIN_Y_N_6           category
RAIN_Y_N_7           category
RAIN_Y_N_8           category
RAIN_Y_N_9           category
RAIN_Y_N_10          category
Length: 183, dtype: object

In [515]:
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression

date_attribs = ['Date', 'DATE_0', 'DATE_1', 'DATE_2', 'DATE_3', 'DATE_4','DATE_5', 'DATE_6', 'DATE_7', 'DATE_8', 'DATE_9', 'DATE_10']
if 'DATE_0' in list(X_train.columns):
    X_train.drop(columns = date_attribs, inplace = True)
num_attribs = X_train.select_dtypes(exclude=['object', 'category', 'datetime64']).columns
cat_attribs = X_train.select_dtypes(include=['object', 'category', 'datetime64']).columns


numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value = 0)),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value=0)),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_attribs),
        ('cat', categorical_transformer, cat_attribs)
    ])

clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', LinearRegression())])

clf.fit(X_train, y_train_1)

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  Pipeline(memory=None,
                                                           steps=[('imputer',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=0,
                                                                                 missing_values=nan,
                                                                                 strategy='constant',
                                                              

In [512]:
cat_attribs = X_train.select_dtypes(include=['object', 'category', 'datetime64']).columns
X_train[cat_attribs].isna().sum()

Location           0
RAIN_Y_N_0     12403
RAIN_Y_N_1     12410
RAIN_Y_N_2     12484
RAIN_Y_N_3     12460
RAIN_Y_N_4     12445
RAIN_Y_N_5     12511
RAIN_Y_N_6     12519
RAIN_Y_N_7     12553
RAIN_Y_N_8     12531
RAIN_Y_N_9     12582
RAIN_Y_N_10    12568
dtype: int64

In [507]:
list(X_train.columns)

['Location',
 'T_30cm',
 'T_60cm',
 'T_90cm',
 'T_120cm',
 'T_150cm',
 'AWND_0',
 'PGTM_0',
 'PRCP_0',
 'TMAX_0',
 'TMIN_0',
 'WDF2_0',
 'WDF5_0',
 'WSF2_0',
 'WSF5_0',
 'WT01_0',
 'WT02_0',
 'WT03_0',
 'WT06_0',
 'WT08_0',
 'AWND_1',
 'PGTM_1',
 'PRCP_1',
 'TMAX_1',
 'TMIN_1',
 'WDF2_1',
 'WDF5_1',
 'WSF2_1',
 'WSF5_1',
 'WT01_1',
 'WT02_1',
 'WT03_1',
 'WT06_1',
 'WT08_1',
 'AWND_2',
 'PGTM_2',
 'PRCP_2',
 'TMAX_2',
 'TMIN_2',
 'WDF2_2',
 'WDF5_2',
 'WSF2_2',
 'WSF5_2',
 'WT01_2',
 'WT02_2',
 'WT03_2',
 'WT06_2',
 'WT08_2',
 'AWND_3',
 'PGTM_3',
 'PRCP_3',
 'TMAX_3',
 'TMIN_3',
 'WDF2_3',
 'WDF5_3',
 'WSF2_3',
 'WSF5_3',
 'WT01_3',
 'WT02_3',
 'WT03_3',
 'WT06_3',
 'WT08_3',
 'AWND_4',
 'PGTM_4',
 'PRCP_4',
 'TMAX_4',
 'TMIN_4',
 'WDF2_4',
 'WDF5_4',
 'WSF2_4',
 'WSF5_4',
 'WT01_4',
 'WT02_4',
 'WT03_4',
 'WT06_4',
 'WT08_4',
 'AWND_5',
 'PGTM_5',
 'PRCP_5',
 'TMAX_5',
 'TMIN_5',
 'WDF2_5',
 'WDF5_5',
 'WSF2_5',
 'WSF5_5',
 'WT01_5',
 'WT02_5',
 'WT03_5',
 'WT06_5',
 'WT08_5',
 'AWND

In [357]:
def pre_process(df):
    num_attribs = X_train.select_dtypes(exclude=['object', 'category', 'datetime64']).columns
    cat_attribs = X_train.select_dtypes(include=['object', 'category']).columns
    date_attribs = X_train.select_dtypes(include=['datetime64']).columns
    
    #numerical preprocessing
    num_imputer = SimpleImputer(strategy='constant', fill_value = 0)
    scaler = StandardScaler()
    for col in num_attribs:
        df[col] = num_imputer.fit_transform(df[col].values.reshape(-1, 1))
        df[col] = scaler.fit_transform(df[col].values.reshape(-1, 1))
        
    cat_imputer = SimpleImputer(strategy='constant', fill_value='missing')
    ohe = OneHotEncoder(sparse=False, handle_unknown="ignore")
    for col in cat_attribs:
        df[col] = cat_imputer.fit_transform(df[col].to_numpy().reshape(-1,1))
        ohe_df = pd.DataFrame(ohe.fit_transform(X_train[['Location']]))
        df = df.join(ohe_df, rsuffix = '_' + col)
    return df

In [424]:
%%time 
start = time()
np.random.seed(42)
full_pipeline_with_predictor = Pipeline([
        #('date_pipeline', date_pipeline),
        ('num_pipeline', num_pipeline),
        ('cat_pipeline', cat_pipeline),

    ])
model = full_pipeline_with_predictor.fit(X_train, y_train)

      Location       Date    T_30cm    T_60cm    T_90cm   T_120cm   T_150cm  \
34470   CAF163 2015-11-06 -0.121350 -0.091090  0.400445 -2.261333  0.844250   
22692   CAF125 2013-04-11 -0.422866 -0.594780 -0.705230 -0.652476 -0.619244   
33962   CAF163 2014-06-16  0.611319  0.476488  0.396327  0.293910  0.272200   
62669   CAF314 2014-11-23 -1.094794 -0.979342 -0.678463 -0.343008  0.124182   
49231   CAF237 2007-10-05  0.065095  0.573555  0.801948  0.965844  1.087371   
...        ...        ...       ...       ...       ...       ...       ...   
67797   CAF351 2015-05-09  0.589470  0.367067  0.124541 -0.001836  0.009057   
43182   CAF209 2014-08-06  1.673179  1.780717  1.607011  1.479258  1.225616   
7727    CAF031 2015-02-21 -0.571147 -0.666786 -0.678463 -0.676136 -0.705051   
24382   CAF129 2013-02-03 -1.021528 -0.891276 -0.863772 -1.031031 -0.633545   
24175   CAF129 2012-06-07  0.301064  0.412953  0.262493  0.175611 -2.182846   

          DATE_0    AWND_0    PGTM_0  ...  RAIN_Y_N

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [358]:
pre_process(X_train)

,Location,Date,T_30cm,T_60cm,T_90cm,T_120cm,T_150cm,DATE_0,AWND_0,PGTM_0,...,32_RAIN_Y_N_10,33_RAIN_Y_N_10,34_RAIN_Y_N_10,35_RAIN_Y_N_10,36_RAIN_Y_N_10,37_RAIN_Y_N_10,38_RAIN_Y_N_10,39_RAIN_Y_N_10,40_RAIN_Y_N_10,41_RAIN_Y_N_10
34470,CAF163,2015-11-06,-0.121350,-0.091090,0.400445,-2.261333,0.844250,2015-11-06,0.633891,-0.497094,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22692,CAF125,2013-04-11,-0.422866,-0.594780,-0.705230,-0.652476,-0.619244,2013-04-11,0.793195,-0.497094,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
33962,CAF163,2014-06-16,0.611319,0.476488,0.396327,0.293910,0.272200,2014-06-16,0.581583,-0.497094,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
62669,CAF314,2014-11-23,-1.094794,-0.979342,-0.678463,-0.343008,0.124182,2014-11-23,2.070001,-0.497094,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49231,CAF237,2007-10-05,0.065095,0.573555,0.801948,0.965844,1.087371,2007-10-05,-0.642914,1.646310,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67797,CAF351,2015-05-09,0.589470,0.367067,0.124541,-0.001836,0.009057,2015-05-09,-0.747531,-0.497094,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43182,CAF209,2014-08-06,1.673179,1.780717,1.607011,1.479258,1.225616,2014-08-06,-0.535919,-0.497094,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
7727,CAF031,2015-02-21,-0.571147,-0.666786,-0.678463,-0.676136,-0.705051,2015-02-21,0.474588,-0.497094,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24382,CAF129,2013-02-03,-1.021528,-0.891276,-0.863772,-1.031031,-0.633545,2013-02-03,-0.747531,-0.497094,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [310]:
ohe = OneHotEncoder(sparse=False, handle_unknown="ignore")
ohe_df = pd.DataFrame(ohe.fit_transform(X_train[['Location']]))
X_train = X_train.join(ohe_df, rsuffix = '_Location')
X_train

,Location,Date,T_30cm,T_60cm,T_90cm,T_120cm,T_150cm,DATE_0,AWND_0,PGTM_0,...,32_Location,33_Location,34_Location,35_Location,36_Location,37_Location,38_Location,39_Location,40_Location,41_Location
34470,CAF163,2015-11-06,-0.121350,-0.091090,0.400445,-2.261333,0.844250,2015-11-06,0.633891,-0.497094,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22692,CAF125,2013-04-11,-0.422866,-0.594780,-0.705230,-0.652476,-0.619244,2013-04-11,0.793195,-0.497094,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
33962,CAF163,2014-06-16,0.611319,0.476488,0.396327,0.293910,0.272200,2014-06-16,0.581583,-0.497094,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
62669,CAF314,2014-11-23,-1.094794,-0.979342,-0.678463,-0.343008,0.124182,2014-11-23,2.070001,-0.497094,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49231,CAF237,2007-10-05,0.065095,0.573555,0.801948,0.965844,1.087371,2007-10-05,-0.642914,1.646310,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67797,CAF351,2015-05-09,0.589470,0.367067,0.124541,-0.001836,0.009057,2015-05-09,-0.747531,-0.497094,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
43182,CAF209,2014-08-06,1.673179,1.780717,1.607011,1.479258,1.225616,2014-08-06,-0.535919,-0.497094,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
7727,CAF031,2015-02-21,-0.571147,-0.666786,-0.678463,-0.676136,-0.705051,2015-02-21,0.474588,-0.497094,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24382,CAF129,2013-02-03,-1.021528,-0.891276,-0.863772,-1.031031,-0.633545,2013-02-03,-0.747531,-0.497094,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [290]:
imputer = SimpleImputer(strategy='constant', fill_value = 0)
X_train['WT08_10'] = imputer.fit_transform(X_train['WT08_10'].values.reshape(-1, 1))
X_train['WT08_10']

34470    0.0
22692    0.0
33962    0.0
62669    0.0
49231    0.0
        ... 
67797    0.0
43182    0.0
7727     0.0
24382    0.0
24175    0.0
Name: WT08_10, Length: 60844, dtype: float64

In [490]:
list(X_train.columns)

['Location',
 'Date',
 'T_30cm',
 'T_60cm',
 'T_90cm',
 'T_120cm',
 'T_150cm',
 'DATE_0',
 'AWND_0',
 'PGTM_0',
 'PRCP_0',
 'TMAX_0',
 'TMIN_0',
 'WDF2_0',
 'WDF5_0',
 'WSF2_0',
 'WSF5_0',
 'WT01_0',
 'WT02_0',
 'WT03_0',
 'WT06_0',
 'WT08_0',
 'DATE_1',
 'AWND_1',
 'PGTM_1',
 'PRCP_1',
 'TMAX_1',
 'TMIN_1',
 'WDF2_1',
 'WDF5_1',
 'WSF2_1',
 'WSF5_1',
 'WT01_1',
 'WT02_1',
 'WT03_1',
 'WT06_1',
 'WT08_1',
 'DATE_2',
 'AWND_2',
 'PGTM_2',
 'PRCP_2',
 'TMAX_2',
 'TMIN_2',
 'WDF2_2',
 'WDF5_2',
 'WSF2_2',
 'WSF5_2',
 'WT01_2',
 'WT02_2',
 'WT03_2',
 'WT06_2',
 'WT08_2',
 'DATE_3',
 'AWND_3',
 'PGTM_3',
 'PRCP_3',
 'TMAX_3',
 'TMIN_3',
 'WDF2_3',
 'WDF5_3',
 'WSF2_3',
 'WSF5_3',
 'WT01_3',
 'WT02_3',
 'WT03_3',
 'WT06_3',
 'WT08_3',
 'DATE_4',
 'AWND_4',
 'PGTM_4',
 'PRCP_4',
 'TMAX_4',
 'TMIN_4',
 'WDF2_4',
 'WDF5_4',
 'WSF2_4',
 'WSF5_4',
 'WT01_4',
 'WT02_4',
 'WT03_4',
 'WT06_4',
 'WT08_4',
 'DATE_5',
 'AWND_5',
 'PGTM_5',
 'PRCP_5',
 'TMAX_5',
 'TMIN_5',
 'WDF2_5',
 'WDF5_5',
 'WSF2_5

In [422]:
y_train_1 = y_train_1.fillna(0)

In [423]:
y_train_1.isna().sum()

0